# Package and distribution identification in 24 hour clusters

Goal: 
    
    Determine package contribution to the 24 hour load profile clusters and which clusters each package belongs to most.

Purpose:

    Identify differences in load profiles.

## Steps:

A. Processing data for 24 hour analyses:  
B. Clustering   
C. Given clusters of 24 hour load profiles (Model 2, 10 minute data) determine:
    
    1. For each of the 225 district clusters, which packages contribute to the cluster (ie. cluster 1 is composed of 10% psn 56, 4% of 63, 3% of 62, etc.)
    2. For each package, determine which clusters the package belongs to most (ie. psn 56 belongs 50% to cluster 1, 30% cluster 3, 2% to cluster 223, etc. This might suggest that this package has mostly steady single load behavior, with a couple conditions of step size shifts.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
from sklearn import metrics

from sqlalchemy import create_engine
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib
from TurbineTimeSeries.storage import MachineDataStore
#from TurbineTimeSeries.transformations import PCA, StandardScaler, DropCols, DropSparseCols, LeftJoin

In [2]:
model_number = 2

store = MachineDataStore('.config')

model_data_hr = (store.query(model_number,'1hr')
                 .not_null(['timestamp','psn'])
                 .exclude_psn([44,52,54,70])
                 .execute())

model_data_min = (store.query(model_number,'10min')
                  .not_null(['timestamp','psn'])
                  .exclude_psn([44,52,54,70])
                  .execute())

model_data_min.head()

,id,lo_c_dp1,f_c_dp1,f_c_dp2,f_c_dp5,pe_c_dt1,g_c_dt1,g_c_dt2,lo_c_dt5,c_dt5_1,...,v_d_2b,lo_t5,g_t2,f_t1,sum_enr,g_t3,g_pct1,f_pos1,timestamp,psn
0,695650,0.227756,-0.076920,-0.096149,0.168523,0.543333,0.402,0.440,0.554118,0.051556,...,0.183143,0.741739,0.831429,0.5030,5199.398481,0.6004,0.16508,0.335737,2017-01-02 13:49:59.999998,56
1,695651,0.219428,-0.076920,-0.144226,0.168526,0.542222,0.396,0.436,0.555294,0.047722,...,0.183143,0.741739,0.829714,0.5005,5199.511899,0.6068,0.16124,0.335336,2017-01-02 14:00:00.000003,56
2,695652,0.227756,-0.115381,-0.144249,0.167950,0.538889,0.402,0.446,0.552941,0.048000,...,0.183143,0.741304,0.833143,0.5030,5199.625316,0.6012,0.17775,0.335737,2017-01-02 14:09:59.999997,56
3,695653,0.223726,-0.076920,-0.096149,0.168138,0.535556,0.400,0.450,0.556471,0.042389,...,0.184057,0.742609,0.834286,0.5030,5199.738734,0.6000,0.16568,0.334535,2017-01-02 14:20:00.000002,56
4,695654,0.229772,-0.115381,-0.096149,0.168523,0.543333,0.396,0.442,0.556471,0.044500,...,0.184057,0.742609,0.832000,0.5025,5199.852658,0.5988,0.16390,0.335336,2017-01-02 14:29:59.999996,56


In [3]:
freq = '10min'
model_data = model_data_min
len(model_data)

1602326

## A. Processing data for 24 hour analyses:

    ● Drop sparse packages without any complete days 
    ● Drop engine starts and engine hour tags
    ● Drop records with any nulls 
    ● Standardize features to 0 mean and unit variance 
    ● PCA on sensor data, each row a point in time with 70 features The top eigenvectors for Model 2, differing from Model 1, correspond to temperature (Eigenvector 1) and actuator states, which correspond to the positions of the various valves throughout the package (Eigenvector 2). Eigenvectors 3 and 4 are less clear cut, but are related to pressure. 

In [4]:
skipped_cols = ['sum_esn','sum_eng_st', 'sum_eng_h']
index_cols = ['id','timestamp','psn']
data_cols = [c for c in model_data.columns if (c not in index_cols) and (c not in skipped_cols)]

In [5]:
missing_values = model_data.isnull().sum().sort_values()
sparse_cols = [x for x in missing_values.index if missing_values[x] > 30000]
clean_data_cols = [x for x in data_cols if x not in sparse_cols]

In [6]:
data = model_data[index_cols + clean_data_cols].dropna().reset_index()
clean_data = StandardScaler().fit_transform(data[clean_data_cols])
print('Once cleaned, the shape of the data is: ', clean_data.shape)

pca =  PCA().fit(clean_data)
reduced = pca.transform(clean_data)

Once cleaned, the shape of the data is:  (1602282, 70)


In [7]:
eigenvectors_to_segment = 4
daily_segments = []
mapping = []

for i in range(eigenvectors_to_segment):
    daily_segments.append([])
    
for psn in data['psn'].sort_values().unique():
    psn_data = pd.DataFrame(data[(data['psn'] == psn)])
    
    psn_data['iso'] = psn_data['timestamp'].apply(lambda x: x.isocalendar())
    psn_data['week'] = psn_data['timestamp'].apply(lambda x: (x.isocalendar()[0],x.isocalendar()[1]))
    psn_data['time'] = psn_data['timestamp'].apply(lambda x: x.time())

    complete_days = (psn_data.groupby(by=['iso']).count()['id'] == 144)
    psn_data['complete_day'] = psn_data['iso'].apply(lambda x: complete_days[x])
    psn_data = psn_data[psn_data['complete_day'] == True]
     
    for d in psn_data['iso'].unique():
        daily_data = (psn_data[psn_data['iso'] == d]).sort_values(by='time')
        idx = daily_data.index
        ids = list(daily_data['id'])
        mapping.append({'psn': psn, 'iso_day': d, 'ids': ids})
        
        for i in range(eigenvectors_to_segment):
            daily_segments[i].append([x for x in reduced[idx,i]])

In [8]:
print('An example of what the mapping dictionary will look like: \n', len(mapping), mapping[0])

An example of what the mapping dictionary will look like: 
 9819 {'psn': 34, 'iso_day': (2015, 49, 6), 'ids': [1211745, 1211746, 1211747, 1211748, 1211749, 1211750, 1211751, 1211752, 1211753, 1211754, 1211755, 1211756, 1211757, 1211758, 1211759, 1211760, 1211761, 1211762, 1211763, 1211764, 1211765, 1211766, 1211767, 1211768, 1211769, 1211770, 1211771, 1211772, 1211773, 1211774, 1211775, 1211776, 1211777, 1211778, 1211779, 1211780, 1211781, 1211782, 1211783, 1211784, 1211785, 1211786, 1211787, 1211788, 1211789, 1211790, 1211791, 1211792, 1211793, 1211794, 1211795, 1211796, 1211797, 1211798, 1211799, 1211800, 1211801, 1211802, 1211803, 1211804, 1211805, 1211806, 1211807, 1211808, 1211809, 1211810, 1211811, 1211812, 1211813, 1211814, 1211815, 1211816, 1211817, 1211818, 1211819, 1211820, 1211821, 1211822, 1211823, 1211824, 1211825, 1211826, 1211827, 1211828, 1211829, 1211830, 1211831, 1211832, 1211833, 1211834, 1211835, 1211836, 1211837, 1211838, 1211839, 1211840, 1211841, 1211842, 1211843

## B. Clustering

    To understand these daily behaviors better, we ran the K-Means clustering algorithm on all 24 hour segments from all packages. 
    
Types of patterns seen:
    
    ● A relatively stable daily fluctuation. This pattern represents normal operating and dominates most of the clusters. Each cluster represents a different load on the package. We have found each package typically has a high and low operating modes.
    ● Normal operating interrupted by one or more jumps between operating modes.
    ● Normal operating interrupted by a quick, sharp spike. This pattern represents transient states as defined in Modeling->Analytic Approach.

In [9]:
sortedLabels = {}
for j, daily_coefficients in enumerate(daily_segments):
    i = 0
    to_be_clustered = daily_coefficients
    n_clusters=225

    cluster = KMeans(n_clusters)
    cluster.fit(to_be_clustered)
    for label in cluster.labels_:
        mapping[i]['label_eig'+str(j+1)] = label
        i += 1
    label_df = pd.DataFrame(cluster.labels_)
    label_counts = label_df[0].value_counts().sort_values(ascending=False)
    labels_sorted_by_freq = list(label_counts.keys())
    sortedLabels['label_eig'+str(j+1)] = dict(zip(labels_sorted_by_freq, range(225)))

## C. Given clusters of 24 hour load profiles (Model 2, 10 minute data) determine:
    
    1. For each of the 225 district clusters, which packages contribute to the cluster (ie. cluster 1 is composed of 10% psn 56, 4% of 63, 3% of 62, etc.)
    2. For each package, determine which clusters the package belongs to most (ie. psn 56 belongs 50% to cluster 1, 30% cluster 3, 2% to cluster 223, etc. This might suggest that this package has mostly steady single load behavior, with a couple conditions of step size shifts.
    
    
But first, we need to fill in the associated cluster results & cluster result frequency order into the original dataset.

In [10]:
ind = []
eig1 = []
eig2 = []
eig3 = []
eig4 = []
for i  in range(len(mapping)):
    indexes = mapping[i]['ids']
    ind.extend(mapping[i]['ids'])
    eig1.extend([mapping[i]['label_eig1']]*len(indexes))
    eig2.extend([mapping[i]['label_eig2']]*len(indexes))
    eig3.extend([mapping[i]['label_eig3']]*len(indexes))
    eig4.extend([mapping[i]['label_eig4']]*len(indexes))
clusterLabels = pd.DataFrame({'id': ind, 'eig1': eig1, 'eig2': eig2, 'eig3': eig3, 'eig4': eig4})
clusterLabels.head()

,eig1,eig2,eig3,eig4,id
0,24,157,34,90,1211745
1,24,157,34,90,1211746
2,24,157,34,90,1211747
3,24,157,34,90,1211748
4,24,157,34,90,1211749


In [11]:
# Merge to dataframe data
print('Shape of data: ', data.shape, '\nShape of clusterLabels: ', clusterLabels.shape)
data.head()

Shape of data:  (1602282, 74) 
Shape of clusterLabels:  (1413936, 5)


,index,id,timestamp,psn,lo_c_dp1,f_c_dp1,f_c_dp2,f_c_dp5,pe_c_dt1,g_c_dt1,...,f_p7,f_p1,v_d_2b,lo_t5,g_t2,f_t1,sum_enr,g_t3,g_pct1,f_pos1
0,0,695650,2017-01-02 13:49:59.999998,56,0.227756,-0.076920,-0.096149,0.168523,0.543333,0.402,...,15.348557,0.647267,0.183143,0.741739,0.831429,0.5030,5199.398481,0.6004,0.16508,0.335737
1,1,695651,2017-01-02 14:00:00.000003,56,0.219428,-0.076920,-0.144226,0.168526,0.542222,0.396,...,15.360575,0.648279,0.183143,0.741739,0.829714,0.5005,5199.511899,0.6068,0.16124,0.335336
2,2,695652,2017-01-02 14:09:59.999997,56,0.227756,-0.115381,-0.144249,0.167950,0.538889,0.402,...,15.348557,0.647773,0.183143,0.741304,0.833143,0.5030,5199.625316,0.6012,0.17775,0.335737
3,3,695653,2017-01-02 14:20:00.000002,56,0.223726,-0.076920,-0.096149,0.168138,0.535556,0.400,...,15.336537,0.646761,0.184057,0.742609,0.834286,0.5030,5199.738734,0.6000,0.16568,0.334535
4,4,695654,2017-01-02 14:29:59.999996,56,0.229772,-0.115381,-0.096149,0.168523,0.543333,0.396,...,15.348557,0.647267,0.184057,0.742609,0.832000,0.5025,5199.852658,0.5988,0.16390,0.335336


In [12]:
labeledData = pd.merge(data, clusterLabels, on='id')
labeledData['eig1_order'] = labeledData['eig1'].map(sortedLabels['label_eig1'])
labeledData['eig2_order'] = labeledData['eig2'].map(sortedLabels['label_eig2'])
labeledData['eig3_order'] = labeledData['eig3'].map(sortedLabels['label_eig3'])
labeledData['eig4_order'] = labeledData['eig4'].map(sortedLabels['label_eig4'])
labeledData.head()

,index,id,timestamp,psn,lo_c_dp1,f_c_dp1,f_c_dp2,f_c_dp5,pe_c_dt1,g_c_dt1,...,g_pct1,f_pos1,eig1,eig2,eig3,eig4,eig1_order,eig2_order,eig3_order,eig4_order
0,0,695650,2017-01-02 13:49:59.999998,56,0.227756,-0.076920,-0.096149,0.168523,0.543333,0.402,...,0.16508,0.335737,79,214,188,150,0,1,18,43
1,1,695651,2017-01-02 14:00:00.000003,56,0.219428,-0.076920,-0.144226,0.168526,0.542222,0.396,...,0.16124,0.335336,79,214,188,150,0,1,18,43
2,2,695652,2017-01-02 14:09:59.999997,56,0.227756,-0.115381,-0.144249,0.167950,0.538889,0.402,...,0.17775,0.335737,79,214,188,150,0,1,18,43
3,3,695653,2017-01-02 14:20:00.000002,56,0.223726,-0.076920,-0.096149,0.168138,0.535556,0.400,...,0.16568,0.334535,79,214,188,150,0,1,18,43
4,4,695654,2017-01-02 14:29:59.999996,56,0.229772,-0.115381,-0.096149,0.168523,0.543333,0.396,...,0.16390,0.335336,79,214,188,150,0,1,18,43


In [15]:
labeledData.to_csv('./package_identification_24hourclusters/dataLabeledWith24HourClusters.csv', index=False)